In [ ]:
from asim.dataset.scene.scene_builder import ArrowSceneBuilder
from asim.dataset.scene.scene_filter import SceneFilter
from nuplan.planning.utils.multithreading.worker_sequential import Sequential


In [ ]:
import os, psutil
import numpy as np

def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")


num_scenes = 100
split = "nuplan_mini_val"

# log_names = ["2021.06.07.12.54.00_veh-35_01843_02314"]
scene_tokens = None
# scene_tokens = ["1b48bd8c60b3790a"]
log_names = None

scene_filter = SceneFilter(split_names=[split], log_names=log_names, scene_tokens=scene_tokens, duration_s=8.1, history_s=1.0)
scene_builder = ArrowSceneBuilder("/home/daniel/asim_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

np.random.seed(42)
scenes = list(np.random.choice(scenes, size=num_scenes, replace=False))

experiment_name = None


In [ ]:
from tqdm import tqdm
from asim.dataset.arrow.conversion import BoxDetectionWrapper, DetectionType
from asim.simulation.gym.demo_gym_env import DemoGymEnv
from asim.simulation.observation.agents_observation import _filter_agents_by_type

from typing import List

from asim.simulation.metrics.sim_agents import get_sim_agents_metrics

from asim.dataset.arrow.conversion import BoxDetection, DetectionType
from asim.dataset.recording.detection.detection import BoxDetectionWrapper

import time

import pandas as pd

action = [1.0, 0.1]  # Placeholder action, replace with actual action logic
env = DemoGymEnv(scenes)

results = []

for scene in tqdm(scenes):

    agent_rollouts = []

    map_api, ego_state, detection_observation, current_scene = env.reset(scene)
    initial_ego_state = ego_state
    agent_rollouts.append(detection_observation.box_detections)

    result = {}
    result["token"] = scene.token
    for i in range(150):
        ego_state, detection_observation, end = env.step(action)
        agent_rollouts.append(detection_observation.box_detections)
        if end:
            break
    result.update(get_sim_agents_metrics(current_scene, agent_rollouts))
    results.append(result)



df = pd.DataFrame(results)
avg_row = df.drop(columns=['token']).mean(numeric_only=True)
avg_row['token'] = 'average'
df = pd.concat([df, pd.DataFrame([avg_row])], ignore_index=True)
df.to_csv("test.csv")
